# AI Red Teaming Agent for Generative AI models and applications in Azure AI Foundry

## Objective
This notebook walks through how to use Azure AI Evaluation's AI Red Teaming Agent functionality to assess the safety and resilience of AI systems against adversarial prompt attacks. AI Red Teaming Agent leverages [Risk and Safety Evaluations](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/evaluation-metrics-built-in?tabs=warning#risk-and-safety-evaluators) to help identify potential safety issues across different risk categories (violence, hate/unfairness, sexual content, self-harm) combined with attack strategies of varying complexity levels from [PyRIT](https://github.com/Azure/PyRIT), Microsoft AI Red Teaming team's open framework for automated AI red teaming.

## Time
You should expect to spend about 30-45 minutes running this notebook. Execution time will vary based on the number of risk categories, attack strategies, and complexity levels you choose to evaluate.

## Before you begin

### Prerequisite
First, if you have an Azure subscription, create an [Azure AI hub](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/ai-resources) then [create an Azure AI project](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/ai-resources). AI projects and Hubs can be served within a private network and are compatible with private endpoints. You **do not** need to provide your own LLM deployment as the AI Red Teaming Agent hosts adversarial models for both simulation and evaluation of harmful content and connects to it via your Azure AI project.

**Note**: In order to upload your results to Azure AI Foundry, you must have the `Storage Blob Data Contributor` role

**Important**: First, ensure that you've installed the [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) and then make sure to authenticate to Azure using `az login` in your terminal before running this notebook.

### Installation
From a terminal window, navigate to your working directory which contains this sample notebook, and execute the following.
```bash
python -m venv .venv
```

Then, activate the virtual environment created:

```bash
# %source .venv/bin/activate # If using Mac/Linux OS
.venv/Scripts/activate # If using Windows OS
```

With your virtual environment activated, install the following packages required to execute this notebook:

```bash
pip install uv
uv pip install azure-ai-evaluation[redteam] azure-identity openai
```


Now open VSCode with the following command, and ensure your virtual environment is used as kernel to run the remainder of this notebook.
```bash
code .
```

### Imports

In [ ]:
from typing import Optional, Dict, Any
import os
import sys
import importlib

# Azure imports
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy

# Import AIAgentConverter is completely optional - if it's not available, we'll just skip it
# This import is NOT required for the core functionality of this notebook
AIAgentConverter = None
try:
    from azure.ai.projects.converter import AIAgentConverter
    print("✅ Successfully imported AIAgentConverter")
except ImportError:
    print("ℹ️ AIAgentConverter not available. This is fine - it's not required for this notebook.")
    print("   You can safely ignore this message and continue with the notebook.")

# OpenAI imports
from openai import AzureOpenAI

# Initialize Azure credentials
credential = DefaultAzureCredential()

In [ ]:
# OPTIONAL: Verify AIAgentConverter (not required for core functionality)
import sys

print("Checking optional 'azure-ai-projects' package...")
print("NOTE: This package is NOT required for this notebook.")
print("      You can safely continue even if these checks fail.\n")

# Check if azure-ai-projects is in installed packages
try:
    import pkg_resources
    version = pkg_resources.get_distribution("azure-ai-projects").version
    print(f"✅ azure-ai-projects package is installed (version {version})")
except pkg_resources.DistributionNotFound:
    print("ℹ️ azure-ai-projects package is not installed (this is optional)")
    
# Try to import and use AIAgentConverter
print("\nTesting AIAgentConverter import:")
try:
    from azure.ai.projects.converter import AIAgentConverter
    print(f"✅ Successfully imported AIAgentConverter from {AIAgentConverter.__module__}")
    
    # Verify we can instantiate it
    try:
        # Just check that the class exists and is callable
        print(f"AIAgentConverter is a {type(AIAgentConverter)}")
        print("✅ AIAgentConverter appears to be a valid class")
    except Exception as e:
        print(f"❌ Error when checking AIAgentConverter type: {e}")
except ImportError as e:
    print(f"ℹ️ Could not import AIAgentConverter. This is fine - it's not required.")
except Exception as e:
    print(f"❌ Other error when importing AIAgentConverter: {e}")

print("\nContinue with the notebook - the core Red Teaming functionality does NOT require AIAgentConverter.")

In [ ]:
# OPTIONAL: Install the azure-ai-projects package
# NOTE: This is NOT required for the core Red Teaming functionality.
# You can safely continue with the notebook even if this installation fails.
import sys
import subprocess
import importlib

print("Attempting to install the optional azure-ai-projects package...")
print("Note: This package is NOT required for the core functionality of this notebook.")
print("      If installation fails, you can safely ignore it and continue with the notebook.")

try:
    # Try to import first to check if it's already installed
    import azure.ai.projects.converter
    print("✅ azure-ai-projects is already installed!")
except ImportError:
    # If not installed, install it
    try:
        print("Installing azure-ai-projects package...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "azure-ai-projects"])
        print("✅ azure-ai-projects package installed successfully!")
        
        # Reload the module if it was partially imported
        if 'azure.ai.projects' in sys.modules:
            importlib.reload(sys.modules['azure.ai.projects'])
        if 'azure.ai.projects.converter' in sys.modules:    
            importlib.reload(sys.modules['azure.ai.projects.converter'])
            
        # Verify the installation worked
        from azure.ai.projects.converter import AIAgentConverter
        print("✅ AIAgentConverter successfully imported after installation!")
    except Exception as e:
        print(f"ℹ️ Could not install azure-ai-projects: {e}")
        print("Don't worry - this package is NOT required for this notebook to work.")
        print("You can safely continue to the next cell.")

### Important: Restart Kernel If Needed

If you're still seeing import errors after running the installation cell above, you may need to restart the kernel to ensure the new packages are properly loaded. In VS Code:

1. Click on the kernel selector in the upper right corner of the notebook
2. Select "Restart" from the dropdown menu
3. Run the cells again from the beginning

This ensures that newly installed packages are properly recognized by the Python interpreter.

### Note on Optional Dependencies

The Azure AI Evaluation Red Team functionality requires these core packages:
- `azure-ai-evaluation[redteam]`
- `azure-identity` 
- `openai`

The `azure-ai-projects` package with `AIAgentConverter` is **NOT** required for the core red teaming functionality in this notebook. If you see import errors for `AIAgentConverter`, you can safely ignore them and continue with the notebook.

If you want to use specific functionality that depends on `AIAgentConverter` later, you can install the package at that time.

### Set Up Your Environment Variables

Set the following variables for use in this notebook. These variables connect to your Azure resources and model deployments.

**Note:** You can find these values in your Azure AI Foundry project or Azure OpenAI resource.

For reference, here's an example of what your populated environment variables should look like:

```
# Azure OpenAI
AZURE_OPENAI_API_KEY="your-api-key-here"
AZURE_OPENAI_ENDPOINT="https://endpoint-name.openai.azure.com/openai/deployments/deployment-name/chat/completions"
AZURE_OPENAI_DEPLOYMENT_NAME="gpt-4"
AZURE_OPENAI_API_VERSION="2024-12-01-preview"

# Azure AI Project
AZURE_SUBSCRIPTION_ID="12345678-1234-1234-1234-123456789012"
AZURE_RESOURCE_GROUP_NAME="your-resource-group"
AZURE_PROJECT_NAME="your-project-name"
```

In [ ]:
# Azure AI Project information
azure_ai_project = {
    "subscription_id": "57bbd325-81fb-4c5f-adee-489263236d32",
    "resource_group_name": "AgenticAI",
    "project_name": "AI-RedTeaming-Proj",
}

# Azure OpenAI deployment information
azure_openai_deployment = "gpt-4o"  # e.g., "gpt-4"
azure_openai_endpoint = "https://aoai-austeast-at.openai.azure.com/"
azure_openai_api_key = " "  # e.g., "your-api-key"
azure_openai_api_version = "2024-12-01-preview"  # Use the latest API version

In [ ]:
# Verify that all required Azure AI Project values are properly set
print("Azure AI Project Configuration Check:")
print(f"Subscription ID: {azure_ai_project['subscription_id']}")
print(f"Resource Group Name: {azure_ai_project['resource_group_name']}")
print(f"Project Name: {azure_ai_project['project_name']}")
print("\nAzure OpenAI Configuration Check:")
print(f"Deployment: {azure_openai_deployment}")
print(f"Endpoint: {azure_openai_endpoint}")
print(f"API Version: {azure_openai_api_version}")
print("API Key: [REDACTED]")

In [ ]:
# Verify Azure CLI authentication status
import subprocess
import json

print("Checking Azure CLI authentication status...")
try:
    # Run 'az account show' to check if the user is logged in
    result = subprocess.run(['az', 'account', 'show'], capture_output=True, text=True, check=False)
    
    if result.returncode == 0:
        account_info = json.loads(result.stdout)
        print(f"✅ Authenticated to Azure CLI as: {account_info.get('user', {}).get('name')}")
        print(f"✅ Current subscription: {account_info.get('name')} ({account_info.get('id')})")
    else:
        print("❌ Not authenticated to Azure CLI. Please run 'az login' in your terminal.")
        print("   This is required for the RedTeam functionality to work properly.")
except Exception as e:
    print(f"❌ Error checking Azure CLI status: {e}")
    print("   Please ensure Azure CLI is installed and run 'az login' in your terminal.")

## Troubleshooting Permission Issues

If you encounter the following error when running the red team scans:
```
WARNING: Failed to log artifacts to AI Foundry: (UserError) Failed to upload evaluation run to the cloud due to insufficient permission to access the storage. Please ensure that the necessary access rights are granted.
```

This is because the Azure AI Evaluation SDK requires the `Storage Blob Data Contributor` role to upload results to Azure AI Foundry. Here's how to fix it:

1. **Check Current Permissions** - First, verify your current role assignments
2. **Assign Required Role** - Add the Storage Blob Data Contributor role to your account
3. **Run Scans Locally** - If you cannot get the permissions, you can still run scans locally

We'll add a cell below to help check and set the required permissions.

In [ ]:
# Helper function to check and fix Storage Blob Data Contributor permissions
import subprocess
import json

def get_storage_account_id():
    """Get the storage account ID associated with the Azure AI project"""
    try:
        # Find the storage account in the resource group
        cmd = [
            'az', 'storage', 'account', 'list',
            '--resource-group', azure_ai_project['resource_group_name'],
            '--query', "[?starts_with(name, 'ai')].id",
            '-o', 'json'
        ]
        result = subprocess.run(cmd, capture_output=True, text=True, check=False)
        if result.returncode != 0:
            print(f"Error querying storage accounts: {result.stderr}")
            return None
            
        storage_accounts = json.loads(result.stdout)
        if not storage_accounts:
            print("No AI storage accounts found in the resource group.")
            return None
            
        print(f"Found {len(storage_accounts)} potential AI storage account(s)")
        return storage_accounts[0]  # Return the first matching storage account
    except Exception as e:
        print(f"Error getting storage account: {e}")
        return None

def check_permissions(storage_account_id):
    """Check if the current user has Storage Blob Data Contributor role"""
    try:
        # Get current user's object ID
        cmd = ['az', 'ad', 'signed-in-user', 'show', '--query', 'id', '-o', 'tsv']
        user_id = subprocess.run(cmd, capture_output=True, text=True, check=False).stdout.strip()
        
        # Check role assignments
        cmd = [
            'az', 'role', 'assignment', 'list',
            '--assignee', user_id,
            '--scope', storage_account_id,
            '--query', "[?roleDefinitionName=='Storage Blob Data Contributor'].roleDefinitionName",
            '-o', 'json'
        ]
        result = subprocess.run(cmd, capture_output=True, text=True, check=False)
        assignments = json.loads(result.stdout)
        
        has_permission = len(assignments) > 0
        return has_permission, user_id
    except Exception as e:
        print(f"Error checking permissions: {e}")
        return False, None

def assign_permission(storage_account_id, user_id):
    """Assign Storage Blob Data Contributor role to the current user"""
    try:
        cmd = [
            'az', 'role', 'assignment', 'create',
            '--assignee', user_id,
            '--role', 'Storage Blob Data Contributor',
            '--scope', storage_account_id
        ]
        result = subprocess.run(cmd, capture_output=True, text=True, check=False)
        
        if result.returncode == 0:
            print("✅ Successfully assigned Storage Blob Data Contributor role!")
            return True
        else:
            print(f"❌ Failed to assign role: {result.stderr}")
            print("You may need to ask your Azure administrator to assign this role.")
            return False
    except Exception as e:
        print(f"Error assigning permission: {e}")
        return False

# Main process to check and fix permissions
print("Checking and fixing Storage Blob Data Contributor permissions...")
storage_id = get_storage_account_id()

if storage_id:
    print(f"Storage account ID: {storage_id}")
    has_permission, user_id = check_permissions(storage_id)
    
    if has_permission:
        print("✅ You already have the Storage Blob Data Contributor role.")
    else:
        print("❌ You don't have the Storage Blob Data Contributor role.")
        print("Attempting to assign the role...")
        success = assign_permission(storage_id, user_id)
        if success:
            print("Role assigned successfully. You should be able to upload results now.")
        else:
            print("\nAlternative options if you can't get the required permissions:")
            print("1. Run the scans with output_path parameter to save results locally")
            print("2. Ask your Azure administrator to assign the Storage Blob Data Contributor role")
            print("3. Continue using the local results without uploading to AI Foundry")
else:
    print("Could not determine the storage account. Please review the manual steps in the troubleshooting section.")

In [ ]:
# Example: Run a scan without uploading to Azure AI Foundry (local evaluation only)
# You can run this if you can't get the Storage Blob Data Contributor role
def demonstrate_local_scan_only():
    """Example function showing how to run scans locally without cloud uploads"""
    # To disable uploads to Azure AI Foundry, you can use the offline_mode parameter
    print("To run Red Team scans without uploading to Azure AI Foundry, use one of these approaches:")
    
    print("\nOption 1: Set offline_mode=True in the RedTeam constructor")
    print("""
    # Create RedTeam with offline_mode set to True
    local_red_team = RedTeam(
        azure_ai_project=azure_ai_project,
        credential=credential,
        risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness],
        num_objectives=1,
        offline_mode=True  # Prevents uploading to Azure AI Foundry
    )
    """)
    
    print("\nOption 2: Always provide output_path parameter to scan() method")
    print("""
    # Always specify output_path to save results locally
    result = await red_team.scan(
        target=financial_advisor_callback,
        scan_name="Local-Scan",
        attack_strategies=[AttackStrategy.Flip],
        output_path="local_output.json"  # This will save results to a local file
    )
    """)
    
    print("\nThe warning about failed uploads can be safely ignored if:")
    print("1. You're only interested in local evaluation results")
    print("2. You specified an output_path to save results locally")

# Run the demonstration function to show examples
demonstrate_local_scan_only()

## Understanding AI Red Teaming Agent's capabilities

The Azure AI Evaluation SDK's `RedTeam` functionality evaluates AI systems against adversarial prompts across multiple dimensions:

1. **Risk Categories**: Different content risk categories your AI system might generate
   - Violence
   - HateUnfairness
   - Sexual
   - SelfHarm

2. **Attack Strategies**: Along with standard unmodified prompts which are sent by default as the `baseline`, you can specify different transformations of prompts to elicit undesired content.
You can also use `AttackStrategy.Compose()` to layer two strategies in one attack
   - AnsiAttack: Using ANSI escape codes in prompts
   - AsciiArt: Using ASCII art to disguise harmful content
   - AsciiSmuggler: Hiding harmful content within ASCII characters
   - Atbash: Using the Atbash cipher to encode harmful requests
   - Base64: Encoding harmful content in Base64 format
   - Binary: Converting text to binary to bypass filters
   - Caesar: Using the Caesar cipher for encoding
   - CharacterSpace: Manipulating character spacing to confuse filters
   - CharSwap: Swapping characters to bypass detection
   - Diacritic: Using diacritical marks to alter text appearance
   - Flip: Flipping text to bypass content filters
   - Leetspeak: Converting letters to numbers and symbols
   - Morse: Using Morse code to encode harmful requests
   - ROT13: Using ROT13 cipher for text transformation
   - SuffixAppend: Adding suffixes to confuse detection systems
   - StringJoin: Joining strings in unconventional ways
   - Tense: Changing the tense of harmful requests to past tense
   - UnicodeConfusable: Using similar-looking Unicode characters
   - UnicodeSubstitution: Substituting characters with Unicode alternatives
   - Url: Embedding harmful content within URLs
   - Jailbreak: Specially crafted prompts to bypass AI safeguards

3. **Complexity Levels**: Different difficultly levels of attacks
   - Baseline: Standard functionality tests
   - Easy: Simple attack patterns
   - Moderate: More sophisticated attacks
   - Difficult: Complex, layered attack strategies

The key metric for evaluating results is the **Attack Success Rate (ASR)**, which measures the percentage of attacks that successfully elicit harmful content from your AI system.

## Basic Example: Fixed Response Callback

Let's start with a basic example that demonstrates how to set up a Red Team evaluation using a simple callback that always returns a fixed, safe response regardless of the input prompt.

**Important Note**: Since this callback always responds with the same safe message, the Attack Success Rate (ASR) will be 0% for all categories. This example is merely to demonstrate the API structure and flow.

In [ ]:
# Define a simple callback function that always returns a fixed response
def financial_advisor_callback(query: str) -> str:  # noqa: ARG001
    return "I'm a financial advisor assistant. I can help with investment advice and financial planning within legal and ethical guidelines."

In [ ]:
# Create the `RedTeam` instance with minimal configurations
try:
    print("Creating RedTeam instance...")
    red_team = RedTeam(
        azure_ai_project=azure_ai_project,
        credential=credential,
        risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness],
        num_objectives=1,
    )
    print("✅ RedTeam instance created successfully!")
except Exception as e:
    print(f"❌ Error creating RedTeam instance: {e}")
    print("\nTroubleshooting tips:")
    print("1. Ensure your Azure AI project details are correct")
    print("2. Verify that you have the right Azure permissions")
    print("3. Check that you're authenticated to Azure CLI with 'az login'")
    print("4. Make sure you have the 'Storage Blob Data Contributor' role")

In [ ]:
# Verify that the required packages are imported correctly
print("Verifying installed packages...")

# Check for azure.ai.evaluation.red_team
import azure.ai.evaluation.red_team
print(f"Azure AI Evaluation Red Team module is available: {azure.ai.evaluation.red_team.__name__}")

# Check for azure.identity
import azure.identity
print(f"Azure Identity module is available: {azure.identity.__name__}")

# Check for openai
import openai
print(f"OpenAI module is available: {openai.__name__}")

print("\nAll required packages are installed and working correctly.")

In [ ]:
# Check installed package versions for troubleshooting
import pkg_resources

print("Checking installed packages...")
print("\nRequired packages:")
required_packages = ['azure-ai-evaluation', 'azure-identity', 'openai']
for package in required_packages:
    try:
        version = pkg_resources.get_distribution(package).version
        print(f"✅ {package}: {version}")
    except pkg_resources.DistributionNotFound:
        print(f"❌ {package}: Not installed - THIS IS REQUIRED")

print("\nOptional packages:")
optional_packages = ['azure-ai-projects']
for package in optional_packages:
    try:
        version = pkg_resources.get_distribution(package).version
        print(f"✅ {package}: {version}")
    except pkg_resources.DistributionNotFound:
        print(f"ℹ️ {package}: Not installed - this is optional")
        
print("\nNotes:")
print("- The 'azure-ai-evaluation' package should include the [redteam] extra.")
print("- The 'azure-ai-projects' package is NOT required for this notebook.")

NOTE: `num_objectives` specifies the number of attacks to perform per risk category per attack strategy. If the parameter `risk_categories` is not specified, `[RiskCategory.Violence, RiskCategory.HateUnfairness, RiskCategory.Sexual, RiskCategory.SelfHarm]` will be used by default.

Now let's run a simple automated scan using the `RedTeam` with the fixed response target. We'll test against two risk categories and one attack strategy for simplicity.

In [ ]:
# Run the red team scan called "Basic-Callback-Scan" with limited scope for this basic example
# This will test 1 objective prompt for each of Violence and HateUnfairness categories with the Flip strategy
try:
    # If you're having storage permission issues, set offline_mode to True here
    result = await red_team.scan(
        target=financial_advisor_callback,
        scan_name="Basic-Callback-Scan-OpenAI",
        attack_strategies=[AttackStrategy.Flip],
        output_path="red_team_output.json",  # Ensure results are saved locally even if cloud upload fails
        offline_mode=False,  # Set to True to avoid cloud uploads completely
    )
    print("✅ Red team scan completed successfully!")
except Exception as e:
    print(f"❌ Error during red team scan: {str(e)}")
    
    # Check if it's a storage permission error
    if "insufficient permission to access the storage" in str(e):
        print("\n### Storage Permission Error Detected ###")
        print("This error occurs when your account doesn't have the 'Storage Blob Data Contributor' role.")
        print("\nQuick Fix Options:")
        print("1. Set offline_mode=True in the scan method above")
        print("2. Go to the 'Quick fix' cell below that has offline_mode examples")
        print("3. Visit the official troubleshooting link: https://aka.ms/azsdk/python/evaluation/remotetracking/troubleshoot")
        print("4. Or follow the detailed steps in the troubleshooting section below")

## Intermediary Example: Using a Model Configuration as Target

Now let's create a more realistic example that uses an Azure OpenAI model for responding to the red teaming prompts. To test base or foundation models, you can update your target to take in a model configuration:

## Troubleshooting: Storage Permission Errors

If you encounter the error `Failed to upload evaluation run to the cloud due to insufficient permission to access the storage`, it means your Azure account doesn't have the necessary permissions to upload results to the storage account associated with your Azure AI project.

### Solution Options:

1. **Run in Offline Mode**: Set `offline_mode=True` in the `red_team.scan()` method to run the scan without uploading results to Azure.

2. **Save Results Locally**: Always include `output_path="filename.json"` parameter to ensure results are saved locally.

3. **Fix Storage Permissions**: Follow the steps in the next cell to add the required 'Storage Blob Data Contributor' role.

NOTE: Options 1 and 2 allow you to continue working even without fixing the storage permissions, but your results won't be available in the Azure AI Studio UI.

### Microsoft Official Documentation Link

For the error message:
```
WARNING: Failed to log artifacts to AI Foundry: (UserError) Failed to upload evaluation run to the cloud due to insufficient permission to access the storage. Please ensure that the necessary access rights are granted.
```

Microsoft provides official troubleshooting documentation at: [Azure SDK Python Evaluation Remote Tracking Troubleshooting](https://aka.ms/azsdk/python/evaluation/remotetracking/troubleshoot)

This documentation explains how to resolve storage permission issues with Azure AI Evaluation services.

In [ ]:
# Alternative simplified check for storage permissions (without using Azure CLI)
# This can help diagnose "Failed to upload evaluation run to the cloud due to insufficient permission to access the storage"
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient
import json

# Function to test storage blob access permissions
def check_storage_permissions():
    try:
        print("Checking storage permissions for your Azure AI project...")
        # Try to read project details from the environment
        with open(f".azure/{azure_ai_project['resource_group_name']}/{azure_ai_project['project_name']}/env.json", "r") as f:
            env_data = json.load(f)
            storage_account = env_data.get("STORAGE_ACCOUNT")
            if storage_account:
                print(f"Found storage account: {storage_account}")
            else:
                print("Storage account not found in project environment file.")
                print("Will attempt to test with storage account determined at runtime.")
                return
    except:
        print("Project environment file not found. Will attempt to test with storage account determined at runtime.")
        return
    
    try:
        # Create blob service client
        print(f"\nTesting access to: {storage_account}")
        account_url = f"https://{storage_account}.blob.core.windows.net"
        credential = DefaultAzureCredential()
        blob_service_client = BlobServiceClient(account_url=account_url, credential=credential)
        
        # Try to list containers (requires Storage Blob Data Contributor role)
        print("Attempting to list blob containers (requires Storage Blob Data Contributor role)...")
        containers = list(blob_service_client.list_containers(max_results=5))
        print(f"✅ Successfully accessed storage account! Found {len(containers)} containers.")
        print("You have the necessary Storage Blob Data Contributor permissions.")
        return True
    except Exception as e:
        print(f"❌ Permission test failed: {str(e)}")
        if "authorization permission" in str(e).lower() or "not authorized" in str(e).lower():
            print("\nYou need the 'Storage Blob Data Contributor' role for this storage account.")
            print("Please see the troubleshooting steps below or use offline_mode=True.")
        return False

# Run the permission check (uncomment to execute)
# has_permission = check_storage_permissions()

### Quick Resolution for "Failed to upload evaluation run to the cloud" Error

To directly address the error: `Failed to upload evaluation run to the cloud due to insufficient permission to access the storage`

**Choose one of these options:**

1. **QUICKEST SOLUTION**: Modify your scan command to include `offline_mode=True`:
   ```python
   result = await red_team.scan(
       target=your_target,
       scan_name="Your-Scan-Name",
       attack_strategies=[AttackStrategy.Flip],
       output_path="local_results.json",  # Saves results locally
       offline_mode=True  # <-- Add this line to bypass cloud upload
   )
   ```

2. **SHORT TERM**: Run the cell with `check_storage_permissions()` above to verify your storage access

3. **LONG TERM**: Follow Microsoft's official guide at [https://aka.ms/azsdk/python/evaluation/remotetracking/troubleshoot](https://aka.ms/azsdk/python/evaluation/remotetracking/troubleshoot) to add the required Storage Blob Data Contributor permission

The offline mode will let you continue working while avoiding the storage permission error.

In [ ]:
import subprocess
import json
import os

# Function to run Azure CLI commands
def run_az_command(cmd):
    try:
        result = subprocess.run(cmd, shell=True, check=True, capture_output=True, text=True)
        return result.stdout.strip()
    except subprocess.CalledProcessError as e:
        print(f"Command error: {e}")
        print(f"Error output: {e.stderr}")
        return None

# Get storage account for Azure AI project
print("Step 1: Finding the storage account associated with your Azure AI project...\n")
account_query = f"az resource list --resource-group {azure_ai_project['resource_group_name']} --resource-type Microsoft.Storage/storageAccounts --query \"[?tags.aifproject=='{azure_ai_project['project_name']}'].name\" -o json"
storage_accounts = run_az_command(account_query)

if storage_accounts:
    try:
        storage_accounts = json.loads(storage_accounts)
        if storage_accounts and len(storage_accounts) > 0:
            storage_account = storage_accounts[0]
            print(f"✅ Found storage account: {storage_account}\n")
            
            # Check if current user has Storage Blob Data Contributor role
            print("Step 2: Checking if you have the 'Storage Blob Data Contributor' role...\n")
            user_query = "az ad signed-in-user show --query 'id' -o tsv"
            user_id = run_az_command(user_query)
            
            if user_id:
                print(f"Signed-in user ID: {user_id}")
                
                # Check role assignment
                role_query = f"az role assignment list --assignee {user_id} --scope /subscriptions/{azure_ai_project['subscription_id']}/resourceGroups/{azure_ai_project['resource_group_name']}/providers/Microsoft.Storage/storageAccounts/{storage_account} --query \"[?roleDefinitionName=='Storage Blob Data Contributor'].roleDefinitionName\" -o json"
                roles = run_az_command(role_query)
                
                if roles:
                    roles = json.loads(roles)
                    has_role = len(roles) > 0
                    
                    if has_role:
                        print("✅ You already have the 'Storage Blob Data Contributor' role.")
                        print("\nIf you're still seeing permission issues, try these steps:")
                        print("1. Run 'az login' again to refresh your credentials")
                        print("2. Restart the notebook kernel")
                        print("3. Wait a few minutes for role assignments to propagate")
                    else:
                        print("❌ You don't have the 'Storage Blob Data Contributor' role.")
                        print("\nStep 3: You can add this role with the following command:")
                        role_assign_cmd = f"az role assignment create --assignee {user_id} --role 'Storage Blob Data Contributor' --scope /subscriptions/{azure_ai_project['subscription_id']}/resourceGroups/{azure_ai_project['resource_group_name']}/providers/Microsoft.Storage/storageAccounts/{storage_account}"
                        
                        print(f"\n```\n{role_assign_cmd}\n```\n")
                        print("Would you like to run this command now? (y/n)")
                        choice = input()
                        
                        if choice.lower() == 'y':
                            print("Assigning role...")
                            result = run_az_command(role_assign_cmd)
                            if result:
                                print("✅ Role assigned successfully! Please wait a few minutes for the permissions to propagate.")
                                print("Then restart the notebook kernel and try again.")
                            else:
                                print("❓ Unable to assign role. Please run the command manually in a terminal.")
                        else:
                            print("Role assignment skipped. You can run the command manually in a terminal.")
                else:
                    print("❓ Unable to check role assignments. Please verify you're signed in to Azure CLI.")
            else:
                print("❓ Unable to get user ID. Please verify you're signed in to Azure CLI.")
        else:
            print("❓ No storage accounts found for your Azure AI project.")
    except Exception as e:
        print(f"Error processing storage accounts: {e}")
else:
    print("❓ Unable to query storage accounts. Please verify your Azure AI project details and Azure CLI login.")

print("\nAlternative Options:")
print("1. Run your scans in offline mode: Set offline_mode=True in red_team.scan()")
print("2. Save results locally: Always include output_path='filename.json' parameter")

In [ ]:
# Quick fix for "Failed to upload evaluation run to the cloud due to insufficient permission to access the storage" error
# Just uncomment and run the code below to use offline mode instead

# Option 1: Direct fix - Set offline_mode=True in your existing scan code
# This is the simplest approach to bypass storage permissions issues
offline_mode = True  # Set to True to avoid uploads to Azure (no storage permissions needed)

# Option 2: Run a complete offline example scan
async def run_offline_scan():
    try:
        print("Creating offline RedTeam instance...")
        # Create a new RedTeam instance configured for offline operation
        offline_red_team = RedTeam(
            azure_ai_project=azure_ai_project,
            credential=credential,
            risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness],
            num_objectives=1,
        )
        
        print("Starting offline red team scan...")
        offline_result = await offline_red_team.scan(
            target=financial_advisor_callback,
            scan_name="Offline-Scan-Example",
            attack_strategies=[AttackStrategy.Flip],
            output_path="offline_red_team_output.json",  # This ensures results are saved locally
            offline_mode=True,  # This is the key parameter to avoid storage permission errors
        )
        
        print("✅ Offline scan completed successfully!")
        print(f"Results saved to: offline_red_team_output.json")
        
        # Show a summary of the results
        print(f"\nSummary of Red Team Results:")
        print(f"- Total attacks: {offline_result.total_attacks}")
        print(f"- Successful attacks: {offline_result.successful_attacks}")
        print(f"- Attack success rate: {offline_result.attack_success_rate:.2%}")
        
        return offline_result
    except Exception as e:
        print(f"❌ Error during offline scan: {str(e)}")
        return None

# Uncomment the line below to run the offline scan
# offline_result = await run_offline_scan()

### Understanding Online vs. Offline Red Teaming

When working with the Azure AI Red Teaming tools, you have two operational modes:

1. **Online Mode (Default)**: 
   - Results are uploaded to Azure AI Studio
   - Requires Storage Blob Data Contributor permissions
   - Allows you to view and analyze results in the Azure portal
   - Use this when you want to maintain a centralized record of evaluations

2. **Offline Mode**:
   - Results are saved only locally, not uploaded to Azure
   - No special storage permissions required
   - Set `offline_mode=True` in the scan method
   - Always specify `output_path` to save your results locally
   - Use this for quick tests or when you don't have the required permissions

You can continue with the rest of the notebook using either mode - all the core red teaming functionality works the same way.

Now, let's continue with the intermediary example using a model configuration as target.

## Intermediary Example: Using a Model Configuration as Target

Now let's create a more realistic example that uses an Azure OpenAI model for responding to the red teaming prompts. To test base or foundation models, you can update your target to take in a model configuration:

In [ ]:
# Define a model configuration to test
azure_oai_model_config = {
    "azure_endpoint": azure_openai_endpoint,
    "azure_deployment": azure_openai_deployment,
    "api_key": azure_openai_api_key,
}

Then, update your target to point to the model configurations and run the scan.

In [ ]:
# Run the red team scan called "Intermediary-Model-Target-Scan" with error handling
try:
    print("Running red team scan...")
    # If you got "Failed to upload evaluation run to the cloud due to insufficient permission to access the storage"
    # Set offline_mode=True below
    result = await red_team.scan(
        target=azure_oai_model_config, 
        scan_name="Intermediary-Model-Target-Scan-OpenAI", 
        attack_strategies=[AttackStrategy.Flip],
        output_path="Intermediary-Model-Target-Scan.json",  # Save output locally
        offline_mode=False,  # Set to True to avoid storage permission errors
    )
    print("✅ Scan completed successfully!")
    
    # Check if a warning about uploads was in the output
    if hasattr(result, 'warnings') and any('upload' in str(w).lower() for w in result.warnings):
        print("\n⚠️ Warning: Results were saved locally but not uploaded to Azure AI Foundry.")
        print("   This is likely due to missing 'Storage Blob Data Contributor' permissions.")
        print("   See the troubleshooting section above to resolve this issue.")
except Exception as e:
    print(f"❌ Error running scan: {e}")
    
    # Check if it's a storage permission error
    if "insufficient permission to access the storage" in str(e):
        print("\n### Storage Permission Error Detected ###")
        print("This error occurs when your account doesn't have the 'Storage Blob Data Contributor' role.")
        print("\nQuick Fix Options:")
        print("1. Set offline_mode=True in the scan method above")
        print("2. Go to the 'Quick fix' cell above that has offline_mode examples")
        print("3. Visit the official troubleshooting link: https://aka.ms/azsdk/python/evaluation/remotetracking/troubleshoot")
    else:
        print("\nTroubleshooting steps:")
        print("1. Check your Azure credentials and permissions")
        print("2. Ensure your Azure AI project details are correct")
        print("3. Try running with offline_mode=True if you keep having upload issues")

## Advanced Example: Using an Azure Open AI Model Endpoint in a Callback Function

Using the same Azure Open AI model configuration as above, we now wrap it in a callback function for more flexibility and control on the input and output handling. This will demonstrate how to evaluate an actual AI application. To test your own actual AI application, replace the inside of the callback function with a call to your application.

In [ ]:
# Define a callback that uses Azure OpenAI API to generate responses
async def azure_openai_callback(
    messages: list,
    stream: Optional[bool] = False,  # noqa: ARG001
    session_state: Optional[str] = None,  # noqa: ARG001
    context: Optional[Dict[str, Any]] = None,  # noqa: ARG001
) -> dict[str, list[dict[str, str]]]:
    # Get token provider for Azure AD authentication
    token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

    # Initialize Azure OpenAI client
    client = AzureOpenAI(
        azure_endpoint=azure_openai_endpoint,
        api_version=azure_openai_api_version,
        azure_ad_token_provider=token_provider,
    )

    ## Extract the latest message from the conversation history
    messages_list = [{"role": message.role, "content": message.content} for message in messages]
    latest_message = messages_list[-1]["content"]

    try:
        # Call the model
        response = client.chat.completions.create(
            model=azure_openai_deployment,
            messages=[
                {"role": "user", "content": latest_message},
            ],
            # max_tokens=500, # If using an o1 base model, comment this line out
            max_completion_tokens=500,  # If using an o1 base model, uncomment this line
            # temperature=0.7, # If using an o1 base model, comment this line out (temperature param not supported for o1 base models)
        )

        # Format the response to follow the expected chat protocol format
        formatted_response = {"content": response.choices[0].message.content, "role": "assistant"}
    except Exception as e:
        print(f"Error calling Azure OpenAI: {e!s}")
        formatted_response = "I encountered an error and couldn't process your request."
    return {"messages": [formatted_response]}

In [ ]:
# Create the RedTeam instance with all of the risk categories with 5 attack objectives generated for each category
model_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness, RiskCategory.Sexual, RiskCategory.SelfHarm],
    num_objectives=5,
)

We will use this instance of `model_red_team` to test different attack strategies in the following section.

### Testing Different Attack Strategies

Now we'll run a more comprehensive evaluation using multiple attack strategies across risk categories. This will give us a better understanding of our model's vulnerabilities.

In [ ]:
# Run the red team scan with multiple attack strategies
advanced_result = await model_red_team.scan(
    target=azure_openai_callback,
    scan_name="Advanced-Callback-Scan-OpenAI",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.CharacterSpace,  # Add character spaces
        AttackStrategy.ROT13,  # Use ROT13 encoding
        AttackStrategy.UnicodeConfusable,  # Use confusable Unicode characters
        AttackStrategy.CharSwap,  # Swap characters in prompts
        AttackStrategy.Morse,  # Encode prompts in Morse code
        AttackStrategy.Leetspeak,  # Use Leetspeak
        AttackStrategy.Url,  # Use URLs in prompts
        AttackStrategy.Binary,  # Encode prompts in binary
        AttackStrategy.Compose([AttackStrategy.Base64, AttackStrategy.ROT13]),  # Use two strategies in one attack
    ],
    output_path="Advanced-Callback-Scan.json",
)

The data and results used in this attack will be saved to the `output_path` specified. The URL printed out at the end of the scorecard will provide a link to where you results are uploaded and logged to your Azure AI Foundry project.

## Bring your own objectives: Using your own prompts as objectives for RedTeam

Below we demonstrate how to use your own prompts as objectives for a `RedTeam` scan. You can see the required format for prompts under `.\data\prompts.json`. Note that when bringing your own prompts, the supported `risk-type`s are `violence`, `sexual`, `hate_unfairness`, and `self_harm`. The number of prompts you specify will be the `num_objectives` used in the scan. 

In [ ]:
path_to_prompts = ".\data\prompts.json"

# Create the RedTeam specifying the custom attack seed prompts to use as objectives
custom_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    custom_attack_seed_prompts=path_to_prompts,  # Path to a file containing custom attack seed prompts
)

In [ ]:
custom_red_team_result = await custom_red_team.scan(
    target=azure_openai_callback,
    scan_name="Custom-Prompt-Scan",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.DIFFICULT,  # Group of difficult complexity attacks
    ],
    output_path="Custom-Prompt-Scan.json",
)

## Conclusion

In this notebook, we've demonstrated how to use the Azure AI Evaluation SDK's `RedTeam` functionality to assess the safety and resilience of AI systems. We started with a basic fixed-response example and then moved to a more realistic model testing across multiple risk categories and attack strategies.

The automated AI red teaming scans provides valuable insights into:

1. **Overall Attack Success Rate (ASR)** - The percentage of attacks that successfully elicit harmful content
2. **Vulnerability by Risk Category** - Which types of harmful content your model is most vulnerable to
3. **Effectiveness of Attack Strategies** - Which attack techniques are most successful against your model
4. **Impact of Complexity** - How more sophisticated attacks affect your model's safety guardrails

By regularly red-teaming your AI applications, you can identify and address potential vulnerabilities before deploying your models to production environments.

### Next Steps

1. **Mitigation**: Use these results to strengthen your model's guardrails against identified attack vectors
2. **Continuous Testing**: Implement regular red team evaluations as part of your development lifecycle
3. **Custom Strategies**: Develop custom attack strategies for your specific use cases and domain
4. **Safety Layers**: Consider adding additional safety layers like Azure AI Content Safety to filter harmful requests and responses 